<a href="https://colab.research.google.com/github/GreihMurray/NLP-2/blob/master/celticMUT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#importing the libraries
import numpy as np
import pandas as pd

In [ ]:
#mounting google drive
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/'MyDrive'
%ls

Mounted at /content/drive
/content/drive/MyDrive
'10 22 2021.gdoc'
'An Intelligent System for Waste Materials Segregation Using IoT and Deep Learning.pdf'
 baby.jpeg
 bigdata_project.gslides
 breast-cancer.data
'car (1).data'
 checkmodel.h5
 CN2_screenshot.gdoc
 CN_Homework1.gdoc
'CN_Homework 2.gdoc'
 CN_homework3.gdoc
 CN_pipeline.gdoc
'Colab Notebooks'/
'COMPUTER VISION ASSIGNMENT 3.gdoc'
'Copy of Pepsi and Vending Refund Request.gform'@
'Copy of Pepsi and Vending Refund Request OLD.gform'
'Copy of Pepsi and Vending Refunds (Responses) OLD.gsheet'
'Copy of Student Development Bank Deposit Form.gform'@
'CSO Reimbursement Request Form - Pending.gsheet'
'CV - 0121.gdoc'
 CV_FinalProject.gdoc
'Data Science Notes.gdoc'
'Deep Learning Notebooks'/
 DL_Project_GI_Tracking.gslides
 ecoli.data
'Emma Project.gdoc'
'Emma Research Project.gdoc'
'E(mpower) Funds Request Form (File responses)'/
'E(mpower) Funds Request Form.gform'@
'English Blurbs summary.gdoc'
'Example Stories.gdoc'
'final .gslide

In [ ]:
#loading the dataset
train = pd.read_csv('train.tsv',sep='\t',header=None)
train.columns = ['words','tag']
train

,words,tag
0,ansin,N
1,),N
2,tá,N
3,níos,N
4,lú,N
...,...,...
5057054,do,N
5057055,daoine,S
5057056,beaga,N
5057057,.,N


In [ ]:
#convert to str type
train=train.astype(str)

In [ ]:
#find out the maximum word and corresponding pos tag
l = train["words"].str.len()
argmax = np.where(l == l.max())[0]
train.iloc[argmax]

,words,tag
2286166,\tN\nní\tN\ndóigh\tN\nliom\tN\ngo\tN\nfuil\tU\...,N


In [ ]:
#length of the maximum word 
len(train['words'][2286166])

57291

In [ ]:
#bigram calculation
def bigram_calculation(df):
  n_gram = []
  for i in range(0,len(df['words'])-1):
    n_gram.append(df['words'][i]+' '+df['words'][i+1])
  return n_gram

In [ ]:
#print the bigram file
bigram_file=bigram_calculation(train)
bigram_file

['ansin )',
 ') tá',
 'tá níos',
 'níos lú',
 'lú gaeilge',
 'gaeilge ag',
 'ag na',
 'na gardaí',
 'gardaí ná',
 'ná bí',
 'bí ariamh',
 'ariamh ainneoin',
 'ainneoin na',
 'na cearta',
 'cearta .',
 '. níl',
 'níl sé',
 'sé ach',
 'ach roinnt',
 'roinnt seachtainí',
 'seachtainí ó',
 'ó sin',
 'sin a',
 'a tógadh',
 'tógadh fear',
 'fear bocht',
 'bocht a',
 'a tug',
 'tug ainm',
 'ainm gaeilge',
 'gaeilge dóibh',
 'dóibh .',
 '. <S>',
 '<S> socraíodh',
 'socraíodh go',
 'go raibh',
 'raibh gá',
 'gá lena',
 'lena leithéid',
 'leithéid ,',
 ', mar',
 'mar go',
 'go bíonn',
 'bíonn na',
 'na <S>',
 '<S> tá',
 'tá an',
 'an córas',
 'córas bainistíochta',
 'bainistíochta tar',
 'tar éis',
 'éis freastal',
 'freastal go',
 'go maith',
 'maith ar',
 'ar rialtas',
 'rialtas áitiúil',
 'áitiúil na',
 'na éireann',
 'éireann agus',
 'agus leanfaidh',
 'leanfaidh údaráis',
 'údaráis áitiúla',
 'áitiúla ar',
 'ar aghaidh',
 'aghaidh ag',
 'ag brath',
 'brath ar',
 'ar bainisteoirí',
 'bainist

In [ ]:
#convert to list
l = train['tag'][1:].tolist()

In [ ]:
#convert bigram tuple
lis=[]
for i in range(len(bigram_file)):
   lis.append((bigram_file[i],l[i]))

In [ ]:
#convert to unigram list
w = train['words'].tolist()
t = train['tag'].tolist()
liss=[]
for i in range(len(w)):
   liss.append((w[i],t[i]))

In [ ]:
#split into testing and training data
test_set = liss[2286166:]
train_set = liss[:2286166] 

In [ ]:
  # j = df['pos'][1:].tolist()
  # b_gram = [ x + y for x, y in zip(gram, j)]

In [ ]:
# data = data.groupby((np.repeat(np.arange(len(data)), 2)[:len(data)])).agg(' '.join)
# data['pos'] = data['pos'].str[2:]
# data['pos'][2528529] = 'N'

In [ ]:
#print an example sum of the tag in train set
(train['tag']=='N').sum()

4339849

In [ ]:
#check the unique elements
tags = {tag for word,tag in train_set}
print(len(tags))
print(tags)
 
#calculate the total words in the vocabulary
vocab = {word for word,tag in train_set}
print(len(vocab))

5
{'S', 'T', 'N', 'H', 'U'}
78089


In [ ]:
#Emission Probability
def emission_prob(word, tag, train_bag = train_set):
    tag_list = [pair for pair in train_bag if pair[1]==tag]
    count_tag = len(tag_list) #total number of times the passed tag occurred in train_set
    w_given_tag_list = [pair[0] for pair in tag_list if pair[0]==word[0]]
    count_w_given_tag = len(w_given_tag_list) #calculate the total number of times the passed word occurred as the passed tag.
 
    return (count_w_given_tag, count_tag)

In [ ]:
#Transition Probability
def transition_prob(tag2, tag1, train_bag = train_set):
    tags = [pair[1] for pair in train_bag]
    count_t1 = len([t for t in tags if t==t1])
    count_t2 = 0
    for index in range(len(tags)-1):
        if tags[index]==t1 and tags[index+1] == t2:
            count_t2 += 1
    return (count_t2, count_t1)

In [ ]:
#Creating a tXt transition matrix 
tags_matrix = np.zeros((len(tags), len(tags)), dtype='float32')
for i, t1 in enumerate(list(tags)):
    for j, t2 in enumerate(list(tags)): 
        tags_matrix[i, j] = transition_prob(t2, t1)[0]/transition_prob(t2, t1)[1]
 
print(tags_matrix)

[[5.1803041e-02 5.8155401e-05 9.4555312e-01 1.4225706e-03 1.1631080e-03]
 [5.7412062e-02 0.0000000e+00 9.3957287e-01 2.8894471e-03 1.2562814e-04]
 [1.0597006e-01 4.0476127e-03 8.4283340e-01 9.2285983e-03 3.7919797e-02]
 [3.9394267e-02 2.1556370e-04 9.5721060e-01 2.2634189e-03 9.1614574e-04]
 [3.9098892e-02 5.3541789e-05 9.5916098e-01 9.6375222e-04 7.2281418e-04]]


In [ ]:
#convert matrix to dataframe
tags_df = pd.DataFrame(tags_matrix, columns = list(tags), index=list(tags))
display(tags_df)

,S,T,N,H,U
S,0.051803,0.000058,0.945553,0.001423,0.001163
T,0.057412,0.000000,0.939573,0.002889,0.000126
N,0.105970,0.004048,0.842833,0.009229,0.037920
H,0.039394,0.000216,0.957211,0.002263,0.000916
U,0.039099,0.000054,0.959161,0.000964,0.000723


In [ ]:
#define Viterbi algorithm
def Viterbi(words, train_bag = train_set):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
     
    for key, word in enumerate(words):
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['N', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                 
            #calculating the emission and state probabilities
            emission_p = emission_prob(words[key], tag)[0]/emission_prob(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
             
        pmax = max(p)
        # getting the state for the maximum probability
        state_max = T[p.index(pmax)] 
        state.append(state_max)
    return list(zip(words, state))

In [ ]:
# import random
# # Let's test our Viterbi algorithm on a few sample sentences of test dataset
# random.seed(1234)      #define a random seed to get same sentences when run multiple times
 
# # choose random 10 numbers
# rndom = [random.randint(1,len(test_set)) for x in range(10)]
 
# # list of 10 sents on which we test the model
# test_run = [test_set[i] for i in rndom]
 
# # list of tagged words
# test_run_base = [tup for sent in test_run for tup in sent]
 
# # list of untagged words
# test_set = [tup[0] for sent in test_run for tup in sent]a

In [ ]:
#testing on the development test set that we have created. 
test_set = train_set[200:]
test_set
print(test_set[:5])
test_run_base = [tup for tup in test_set]

[('póca', 'N'), ('agam', 'N'), ('cinnte', 'N'), ('ag', 'N'), ('an', 'N')]


In [ ]:
#running the model. We are using only the first 200 to check if we are getting the right accuracy.
tagged_seq = Viterbi(test_set[0:200])
# accuracy
check = [i for i, j in zip(tagged_seq, test_run_base) if i[1] == j[1]] 
#print(tagged_seq)
accuracy = len(check)/len(tagged_seq)  
print('Viterbi Algorithm Accuracy: ',accuracy*100)

[(('póca', 'N'), 'N'), (('agam', 'N'), 'N'), (('cinnte', 'N'), 'N'), (('ag', 'N'), 'N'), (('an', 'N'), 'N'), (('am', 'N'), 'N'), (('sin', 'N'), 'N'), (('ach', 'N'), 'N'), (('ní', 'N'), 'N'), (('raibh', 'N'), 'N'), (('sé', 'N'), 'N'), (('ceangailte', 'N'), 'N'), (('leis', 'N'), 'N'), (('an', 'N'), 'N'), (('idirlíon', 'N'), 'N'), (('ná', 'N'), 'N'), (('an', 'N'), 'N'), (('ríomhphost', 'N'), 'N'), (('a', 'N'), 'N'), (('bí', 'S'), 'S'), (('agam', 'N'), 'N'), (('ag', 'N'), 'N'), (('an', 'N'), 'N'), (('am', 'N'), 'N'), ((',', 'N'), 'N'), (('ríomhphost', 'N'), 'N'), (('de', 'N'), 'N'), (('cuid', 'S'), 'S'), (('an', 'N'), 'N'), (('comhairle', 'S'), 'N'), (('contae', 'N'), 'S'), (('a', 'N'), 'N'), (('bí', 'S'), 'S'), (('agam', 'N'), 'N'), (('agus', 'N'), 'N'), (('ceann', 'N'), 'N'), (('pearsanta', 'N'), 'N'), (('agus', 'N'), 'N'), (('an', 'N'), 'N'), (('aon', 'T'), 'N'), (('pointe', 'S'), 'N'), (('teagmhála', 'N'), 'N'), (('a', 'N'), 'N'), (('bí', 'S'), 'S'), (('leis', 'N'), 'N'), (('an', 'N'),